**Лабораторная работа 1**

**Выполнил: Кирякин Максим**

**Дата:** `3/9/2025`

## Set up

In [1]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
from statsmodels.tsa.stattools import acf

from utils.utils import *

## Part 1

In [2]:
# specification

config = {
    'S0':     100,    # цена актива в начальный момент времени
    'r':      0.05,   # безрисковая ставка
    'q':      0.02,   # дивидендная доходность
    'sigma':  0.3,    # стандартное отклонение
    'gamma':  0.75,   # 
    'T':      1.0,    # 'дата' экспирации контракта
}
n = 12     # число выплат
N_list = np.arange(1000, 10000, 1000)#[1000, 10_000, 50_000] # число симуляций
m_list = np.arange(10, 1000, 100)#[10, 50, 100]          # число шагов дискретизации

In [3]:
paths = simulate_cev(
    S0=config['S0'],
    r=config['r'],
    q=config['q'],
    sigma=config['sigma'],
    gamma=config['gamma'],
    T=config['T'],
    m=100, N=1000
)
plot_mc_paths(paths, num_paths=15)

График `Graphs of 15 random trajectories of CEV process.` был сохранен: logs/graphs/cev_mc.png


In [37]:
# некоторые простые проверки
assert np.allclose(paths[:, 0], config['S0']), "Начальное условие S0 не соблюдается!"
assert np.all(paths >= 0), "Обнаружены отрицательные значения цены!"
assert (acf(np.diff(paths, axis=1)[1, :], nlags=20, fft=True)[1:] < 1e-1).all(), "Есть корреляция приращений!"

In [4]:
# Генерация таблицы
ans = []
for N in tqdm(N_list):
    row = [N]
    for m in m_list:
        paths = simulate_cev(**config, N=N, m=m)
        price, _ = cliquet_price(**config, N=N, n=n, m=m, paths=paths)
        row.append(np.round(price, 2))
    ans.append(row)

pd.DataFrame(ans, columns=['N/m'] + list(m_list)).set_index('N/m')

100%|█████████████████████████████████████████████| 9/9 [00:36<00:00,  4.07s/it]


,10,110,210,310,410,510,610,710,810,910
N/m,,,,,,,,,,
1000,13.40,14.10,14.36,14.39,14.20,14.09,14.48,14.28,14.43,14.45
2000,13.60,14.60,14.38,14.33,14.54,14.18,14.50,14.46,14.37,14.39
3000,13.12,14.43,14.69,14.52,14.25,14.46,14.52,14.51,14.41,14.55
4000,13.26,14.35,14.49,14.60,14.62,14.24,14.48,14.45,14.40,14.31
5000,13.24,14.49,14.34,14.37,14.24,14.37,14.43,14.50,14.45,14.44
6000,13.17,14.37,14.26,14.52,14.52,14.38,14.49,14.54,14.29,14.35
7000,13.41,14.37,14.48,14.44,14.36,14.41,14.42,14.51,14.50,14.50
8000,13.17,14.51,14.46,14.40,14.51,14.30,14.45,14.37,14.42,14.38
9000,13.19,14.47,14.58,14.59,14.36,14.43,14.47,14.46,14.46,14.49


**Выводы:** 